# Generate sample

# 1. Retrieve all Ratio Analyses from CSV file

Specify the path and name of the CSV file. Read the file and display the resulting dataframe.

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

path = './Analyses/'
file = 'analyses_ratios'
type_file = '.csv'

analyses = pd.read_csv(path+file+type_file, index_col = 0)

analyses.head()

,company,current_ratio_2021,current_ratio_2022,quick_ratio_2021,quick_ratio_2022,cash_ratio_2021,cash_ratio_2022,analysis_current_ratio,analysis_quick_ratio,analysis_cash_ratio
0,3M,1.70,1.54,1.08,0.93,0.51,0.38,"In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels."
1,Adobe,1.25,1.11,1.11,1.00,0.55,0.52,"Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquidity position has weakened compared to the previous year. While the quick ratio is still between 1.00 and 1.50, the decrease raises concerns about the ability to cover short-term liabilities with the most liquid assets.","Both in 2021 and 2022, cash exceeds appropriate levels, so short-term resources are not being used adequately. Although the ratio has slightly decreased in 2022, the excess of liquidity persists."
2,Airbnb,1.95,1.86,1.89,1.81,0.95,0.92,"In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events impacting liquidity. While the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","In 2021 and 2022, current assets (excluding inventories and other less liquid assets) are more than sufficient to meet short-term current liabilities. This implies that the company presents an excess of liquidity and there is an opportunity to allocate resources towards liquidity for alternative purposes. Anyway, the ratio has decreased in 2022 compared to 2021.","In both 2021 and 2022, the amount of cash surp

In [2]:
analyses.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company                 50 non-null     object 
 1   current_ratio_2021      50 non-null     float64
 2   current_ratio_2022      50 non-null     float64
 3   quick_ratio_2021        50 non-null     float64
 4   quick_ratio_2022        50 non-null     float64
 5   cash_ratio_2021         50 non-null     float64
 6   cash_ratio_2022         50 non-null     float64
 7   analysis_current_ratio  50 non-null     object 
 8   analysis_quick_ratio    50 non-null     object 
 9   analysis_cash_ratio     50 non-null     object 
dtypes: float64(6), object(4)
memory usage: 4.3+ KB


# 2. Sample generation

Auxiliary function to convert the constraints of each type of ratio to text.

In [3]:
def ratio_categories(ratio_name, ratio_value):
    switch_cases = {
        'Current': {
            lambda x: x < 1.00: "less than 1.00",
            lambda x: 1.00 <= x <= 1.49: "between 1.00 and 1.49",
            lambda x: 1.50 <= x <= 2.00: "between 1.50 and 2.00",
            lambda x: x > 2.00: "greater than 2.00",
        },
        'Quick': {
            lambda x: x < 1.00: "less than 1.00",
            lambda x: 1.00 <= x <= 1.50: "between 1.00 and 1.50",
            lambda x: x > 1.50: "greater than 1.50",
        },
        'Cash': {
            lambda x: x < 0.20: "less than 0.20",
            lambda x: 0.20 <= x <= 0.40: "between 0.20 and 0.40",
            lambda x: x > 0.40: "greater than 0.40",
        },
    }

    if ratio_name in switch_cases:
        for case, message in switch_cases[ratio_name].items():
            if case(ratio_value):
                return message
        else:
            print("Invalid ratio_value for", ratio_name)
    else:
        print("Invalid ratio_name")

Auxiliary function to convert the variations in the ratio from one year to another into text.

In [4]:
def ratio_variation(value_year_x, value_year_y):
    if value_year_x is None or value_year_y is None:
        return "Invalid values: Both values must be provided and not be None."

    if not (isinstance(value_year_x, (int, float)) and isinstance(value_year_y, (int, float))):
        return "Invalid values: Both values must be numbers."

    variation = None

    if value_year_x == value_year_y:
        variation = "remained at"
    elif value_year_x < value_year_y:
        variation = "increased to"
    else:
        variation = "decreased to"

    return variation

Auxiliary function to generate the instruction (the question).

In [5]:
def generate_template(ratio_name, value_year_x, value_year_y, year_x, year_y, category_year_x, category_year_y, variation):
    template_1 = ratio_name+ " ratio of a company is "+value_year_x+" in "+year_x+", placing its value in the category defined as "+category_year_x+". In "+year_y+", "+ratio_name+" ratio has "+variation+" "+value_year_y+", placing its value in the category defined as "+category_year_y+". What would be the "+ratio_name+" ratio analysis for the years "+year_x+" and "+year_y+"?"
    return template_1

Function used to generate the training sample. It iterates through all the ratio analyses in the dataframe and utilizes auxiliary functions to ultimately generate the sample.

In [6]:
def generate_sample(dataframe):
    
    current_ratio_name = 'Current'
    quick_ratio_name = 'Quick'
    cash_ratio_name = 'Cash'
    year_x = '2021'
    year_y = '2022'
    
    dataframes_list = []
    
    for index, row in dataframe.iterrows():
        company = row['company']
        current_ratio_value_year_x = row['current_ratio_2021']
        current_ratio_value_year_y = row['current_ratio_2022']
        category_current_ratio_year_x = ratio_categories(current_ratio_name, current_ratio_value_year_x)
        category_current_ratio_year_y = ratio_categories(current_ratio_name, current_ratio_value_year_y)
        current_ratio_variation = ratio_variation(current_ratio_value_year_x, current_ratio_value_year_y)
        analysis_current_ratio = row['analysis_current_ratio']
        
        current_ratio_instruction = generate_template(
            current_ratio_name,
            "{:.2f}".format(current_ratio_value_year_x),
            "{:.2f}".format(current_ratio_value_year_y),
            year_x,
            year_y,
            category_current_ratio_year_x,
            category_current_ratio_year_y,
            current_ratio_variation
        )
        
        current_ratio_new_row = pd.DataFrame({
            'instruction': [current_ratio_instruction],
            'analysis': [analysis_current_ratio],
            'ratio': [current_ratio_name],
            'company': [company]
        })

        dataframes_list.append(current_ratio_new_row)
        
        quick_ratio_value_year_x = row['quick_ratio_2021']
        quick_ratio_value_year_y = row['quick_ratio_2022']
        category_quick_ratio_year_x = ratio_categories(quick_ratio_name, quick_ratio_value_year_x)
        category_quick_ratio_year_y = ratio_categories(quick_ratio_name, quick_ratio_value_year_y)
        quick_ratio_variation = ratio_variation(quick_ratio_value_year_x, quick_ratio_value_year_y)
        analysis_quick_ratio = row['analysis_quick_ratio']
        
        quick_ratio_instruction = generate_template(
            quick_ratio_name,
            "{:.2f}".format(quick_ratio_value_year_x),
            "{:.2f}".format(quick_ratio_value_year_y),
            year_x,
            year_y,
            category_quick_ratio_year_x,
            category_quick_ratio_year_y,
            quick_ratio_variation
        )
        
        quick_ratio_new_row = pd.DataFrame({
            'instruction': [quick_ratio_instruction],
            'analysis': [analysis_quick_ratio],
            'ratio': [quick_ratio_name],
            'company': [company]
        })

        dataframes_list.append(quick_ratio_new_row)
        
        cash_ratio_value_year_x = row['cash_ratio_2021']
        cash_ratio_value_year_y = row['cash_ratio_2022']
        category_cash_ratio_year_x = ratio_categories(cash_ratio_name, cash_ratio_value_year_x)
        category_cash_ratio_year_y = ratio_categories(cash_ratio_name, cash_ratio_value_year_y)
        cash_ratio_variation = ratio_variation(cash_ratio_value_year_x, cash_ratio_value_year_y)
        analysis_cash_ratio = row['analysis_cash_ratio']
        
        cash_ratio_instruction = generate_template(
            cash_ratio_name,
            "{:.2f}".format(cash_ratio_value_year_x),
            "{:.2f}".format(cash_ratio_value_year_y),
            year_x,
            year_y,
            category_cash_ratio_year_x,
            category_cash_ratio_year_y,
            cash_ratio_variation
        )
        
        cash_ratio_new_row = pd.DataFrame({
            'instruction': [cash_ratio_instruction],
            'analysis': [analysis_cash_ratio],
            'ratio': [cash_ratio_name],
            'company': [company]
        })

        dataframes_list.append(cash_ratio_new_row)
          
    sample = pd.concat(dataframes_list, ignore_index=True)
    return sample
        

Sample generation from the dataframe containing all ratio analyses. Display the resulting dataframe.

In [7]:
sample = generate_sample(analyses)
sample

,instruction,analysis,ratio,company
0,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.",Current,3M
1,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.",Quick,3M
2,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.",Cash,3M
3,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",Current,Adobe
4,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquid

# 3. Store the sample in a CSV file

Specify the path and name of the file containing the sample.

In [8]:
path = './Sample/'
file = 'sample'
type_file = '.csv'

sample.to_csv(path+file+type_file, index=True)